In [ ]:
import numpy as np
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "./all_images_300"]).decode("utf8"))

In [ ]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import glob
import cv2
from matplotlib import pyplot as pp
from matplotlib import colors as pc
import numpy as np
import os

def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

In [ ]:
def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    #resized = cv2.resize(img, (64, 64), cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    #return [path, resized]
    return [path, img]

def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

print('Start pre-processing training set')
train = glob.glob('./all_images_300/train/**/*.jpg') + glob.glob('./all_images_300/additional/**/*.jpg')
#train=glob.glob('../input/train/Type_1/*.jpg')[:5] +glob.glob('../input/train/Type_2/*.jpg')[:5] +glob.glob('../input/train/Type_3/*.jpg')[:5]
#train = glob.glob('./input/roi/train/**/*.jpg')
train = pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train], columns = ['type','image','path']) #limit for Kaggle Demo
train = im_stats(train)
#train = train[train['size'] != '0 0'].reset_index(drop=True) #remove bad images
train_data = normalize_image_features(train['path'])
#train_data = roi(pathtrain)
#np.save('train64x64.npy', train_data, allow_pickle=True, fix_imports=True)

le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
print(le.classes_) #in case not 1 to 3 order
#np.save('train_target64x64.npy', train_target, allow_pickle=True, fix_imports=True)

print('Start pre-processing test set')
test = glob.glob('./all_images_300/test/*.jpg') + glob.glob('./all_images_300/test_stg2/*.jpg')
#test = glob.glob('./input/roi/test/*.jpg')
test = pd.DataFrame([[p.split('/')[3],p] for p in test], columns = ['image','path']) #[::20] #limit for Kaggle Demo
test_data = normalize_image_features(test['path'])
#test_data=roi(pathtest)
#np.save('test64x64.npy', test_data, allow_pickle=True, fix_imports=True)

test_id = test.image.values
#np.save('test_id64x64.npy', test_id, allow_pickle=True, fix_imports=True)

In [ ]:
#Test final architecure with simple cross-validation 70-30 split
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras import backend as K
K.set_image_dim_ordering('th')
K.set_floatx('float32')

import pandas as pd
import numpy as np
np.random.seed(17)

#train_data = np.load('train.npy')
#train_target = np.load('train_target.npy')

x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.3, random_state=17)

def create_model(opt_='adadelta'):
    model = Sequential()
    model.add(Convolution2D(4, 3, 3, activation='relu', dim_ordering='th', input_shape=(3, 300, 300))) #use input_shape=(3, 64, 64)
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Convolution2D(8, 3, 3, activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(12, activation='tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer=opt_, loss='sparse_categorical_crossentropy', metrics=['accuracy']) 
    return model

datagen = ImageDataGenerator(rotation_range=0.3, zoom_range=0.3)
datagen.fit(train_data)

model = create_model()
print(x_train.shape)
print(y_train.shape)
model.fit_generator(datagen.flow(x_train,y_train, batch_size=15, shuffle=True), nb_epoch=50, samples_per_epoch=len(x_train), verbose=2, validation_data=(x_val_train, y_val_train))
#model.fit(x_train,y_train, batch_size=15, nb_epoch=100, verbose=2, validation_data=(x_val_train, y_val_train))

#test_data = np.load('test.npy')
#test_id = np.load('test_id.npy')

pred = model.predict_proba(test_data)
df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission_300.csv', index=False)


print(cpu_count())

In [ ]:
#Reshaped dataset to work with TensorFlow
train_data = train_data.transpose((0, 2, 3, 1))
test_data = test_data.transpose((0, 2, 3, 1))

In [ ]:
# VGG16 fine-tuning, did not work also tried to run on SurfSara
from keras import applications
from keras.models import Sequential, Model
from keras import optimizers
K.set_image_dim_ordering('tf')

x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.3, random_state=17)

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
print('Base-model loaded.')

for layer in base_model.layers:
    layer.trainable = False

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='tanh'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax', name='predictions'))

model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

datagen = ImageDataGenerator(rotation_range=0.3, zoom_range=0.3)
datagen.fit(train_data)

model.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(datagen.flow(x_train,y_train, batch_size=15, shuffle=True), nb_epoch=50, samples_per_epoch=len(x_train), verbose=2, validation_data=(x_val_train, y_val_train))

print('Unfreeze layers!!!')

for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

model.compile(optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train,y_train, batch_size=15, shuffle=True), nb_epoch=50, samples_per_epoch=len(x_train), verbose=2, validation_data=(x_val_train, y_val_train))

pred = model.predict_proba(test_data)
df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission_vgg_first.csv', index=False)


print(cpu_count())